In [1]:
import keras

In [2]:
from keras.models import load_model

In [4]:
model_new = load_model("model_saved/Yaw_model.h5")

In [10]:
! pip install coremltools

     ------------------------------------ 941.9/941.9 kB 312.1 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 5.7/5.7 MB 453.5 kB/s eta 0:00:00
     ------------------------------------ 536.2/536.2 kB 646.9 kB/s eta 0:00:00
  Created wheel for coremltools: filename=coremltools-6.3.0-py3-none-any.whl size=1220319 sha256=9f5affeb5b869357fdee322841580f428095e5ad54925f455bc7aca3e53a4f73
  Stored in directory: c:\users\farhad\appdata\local\pip\cache\wheels\c4\d9\07\21c9bf958992c296738b5e472ff3c06880299998315169d4ac
Successfully built coremltools



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import coremltools

In [10]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [12]:
coreml_model = coremltools.convert("model_saved/Yaw_model.h5")

# coreml_model.visualize_spec()

coreml_model.save("model_saved/Yaw_prediction.mlmodel")

Translating MIL ==> NeuralNetwork Ops: 100%|██████████| 194/194 [00:27<00:00,  7.18 ops/s]


In [14]:
import torch
model_yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model_yolov5.eval()

Using cache found in C:\Users\Farhad/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-7-10 Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): Model(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
           

In [7]:
import cv2

im = cv2.imread("sh/bb/3769.png")
# im =  torch.rand(1,700,700,3) 
tt = torch.from_numpy(im)

In [9]:
traced_model = torch.jit.trace(model_yolov5 , torch.unsqueeze(tt,0))

RuntimeError: Given groups=1, weight of size [32, 3, 6, 6], expected input[1, 383, 494, 3] to have 3 channels, but got 383 channels instead

In [57]:
import coremltools as ct

# Load the ONNX model coremltools.convert
onnx_model = ct.convert('yolov5s.pt')

# Convert the ONNX model to a Core ML model
mlmodel = ct.convert(onnx_model)

ValueError: Unable to determine the type of the model, i.e. the source framework. Please provide the value of argument "source", from one of ["tensorflow", "pytorch", "milinternal"]. Note that model conversion requires the source package that generates the model. Please make sure you have the appropriate version of source package installed. E.g., if you're converting model originally trained with TensorFlow 1.14, make sure you have `tensorflow==1.14` installed.

In [ ]:
mlmodel.save('my_mlmodel.mlmodel')


In [ ]:
model_yolov5.to(torch.device('cpu'))

In [16]:
next(model_yolov5.parameters()).is_cuda

False

In [19]:
! pip install onnx

^C


In [20]:
import onnx

ModuleNotFoundError: No module named 'onnx'

     --------                                 2.8/13.3 MB 17.4 kB/s eta 0:10:00


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Farhad\anaconda3\envs\gputest\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\Farhad\anaconda3\envs\gputest\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\Farhad\anaconda3\envs\gputest\lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
  File "C:\Users\Farhad\anaconda3\envs\gputest\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\Farhad\anaconda3\envs\gputest\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\Farhad\anaconda3\envs\gputest\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\Farhad\anaconda3\envs\gputest\lib\socket.py",

In [45]:
input_names = ['image']
output_names = ['output']

# Export PyTorch model to ONNX
torch.onnx.export(model_yolov5, torch.randn((1, 3, 640, 640)), 'yolov5s.onnx', export_params=True, input_names=input_names, output_names=output_names)

C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:514: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  y = self.model(im, augment=augment, visualize=visualize) if augment or visualize else self.model(im)
C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master\models\yolo.py:64: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.grid[i].shape[2:4] != x[i].shape[2:4]:


================ Diagnostic Run torch.onnx.export version 2.0.0 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
mlmodel = ct.converters.onnx.convert(model=onnx_model_path)

In [13]:
import coremltools as ct

# Load the ONNX model
# onnx_model = onnx.load('yolov5s.onnx')

# Convert ONNX model to Core ML model
# mlmodel = ct.convert('yolov5s.onnx')

# # Save the Core ML model
# mlmodel.save('yolov5s.mlmodel')

In [ ]:
import torch

NameError: name 'torch' is not defined